In [1]:
!pip install vega_datasets

In [2]:
import pandas as pd
import altair as alt
import os 
from vega_datasets import data

In [5]:
# List files in data/Taste/
files = os.listdir('data/Taste/')
# Read in all files with pandas
# Filename as a column
dfs = []
for file in files:
    dfs.append(pd.read_csv('data/Taste/' + file))
    
# Concatenate all files
taste = pd.concat(dfs)
taste

,sweetness,saltiness,sourness,bitterness,savoriness,fattiness,spiciness,id,title,cuisine
0,34.20,100.00,36.03,24.14,48.96,60.28,0.0,759983,Tortilla Lasagna with Swiss Chard,Italian
1,88.34,96.81,100.00,88.04,38.84,33.89,0.0,631051,"Avocado, Mozzarella and Bruschetta Chicken - I...",Italian
2,33.02,100.00,22.62,33.36,79.68,61.05,14900.0,612345,{One Dish} Quinoa Chicken Parmesan Casserole,Italian
3,20.08,100.00,23.24,18.63,66.20,62.56,14900.0,955390,Zucchini & Sweet Potato Noodle Minestrone,Italian
4,100.00,65.23,99.22,49.19,52.86,58.83,0.0,347822,Pasta Primavera,Italian
...,...,...,...,...,...,...,...,...,...,...
495,100.00,14.41,3.44,8.74,10.62,91.17,0.0,552689,Walnut-Rosemary Olive Oil Scones,British
496,19.21,87.47,19.36,15.22,42.11,100.00,0.0,585934,Fish and Chips with Beer Batter,British
497,100.00,66.47,35.33,26.69,61.59,96.59,0.0,603889,Alaskan fish and chips,British
498,79.55,10.68,36.44,49.79,16.66,100.00,0.0,493798,Easy Cheddar Scones,British


In [4]:
#flavor_pivoted_byCuisine = pd.melt(taste, id_vars=['cuisine'], value_vars=['sweetness', 'saltiness','sourness','bitterness','savoriness','fattiness', 'spiciness'])
# Melt with 'cuisine' as key
import numpy as np

flavor_grouped_byCuisine = taste.groupby("cuisine", as_index=False).agg(
    sweetness = ("sweetness", np.mean),
    saltiness = ("saltiness", np.mean),
    sourness = ("sourness", np.mean),
    bitterness = ("bitterness", np.mean),
    savoriness = ("savoriness", np.mean),
    fattiness = ("fattiness", np.mean),
    spiciness = ("spiciness", np.mean)
)

In [7]:
flavor_grouped_byCuisine

,cuisine,sweetness,saltiness,sourness,bitterness,savoriness,fattiness,spiciness
0,African,62.137824,67.217353,34.865824,28.495176,38.197941,66.10300,5.527374e+05
1,American,59.044920,72.916520,41.882140,30.361040,43.979340,67.00252,1.446679e+06
2,British,66.256100,44.988180,23.402340,17.131300,23.229080,74.42786,6.065368e+04
3,Cajun,46.537140,77.776140,34.270740,26.874960,46.474800,68.57418,2.792083e+05
4,Caribbean,62.250940,60.223000,25.631920,22.126660,39.433700,60.23418,6.899148e+05
5,Chinese,46.824080,86.889420,26.197580,33.452680,53.504020,66.59982,8.045560e+05
6,Eastern European,39.229620,95.212700,32.917500,29.876300,58.345860,63.77872,1.999277e+05
7,European,54.305180,78.691680,46.642340,36.208560,45.267140,68.12250,1.050325e+05
8,French,44.684660,88.074600,34.860460,31.167400,44.656160,70.55514,1.559065e+05
9,German,49.035120,72.032500,22.098900,20.779260,38.950700,66.35832,2.003863e+05


In [8]:
## Create bar chart with flavor profiling
## NEED TO UPDATE TO DYNAMICALLY SPIT OUT MULTIPLE CHARTS
input_dropdown_cuisine = alt.binding_select(options=taste.cuisine.unique(), name='Cuisines')
selection = alt.selection_multi(fields=['cuisine'], empty='none')

alt.data_transformers.enable('default', max_rows=None)

color = alt.condition(selection, alt.Color('cuisine:N', legend=None), alt.value('lightgray'))
make_selector = alt.Chart(flavor_grouped_byCuisine).mark_rect().encode(y='cuisine', color=color).add_selection(selection)

# Layering and configuring the components
chart_sweet = alt.Chart(flavor_grouped_byCuisine).mark_bar(
    ).encode(
        x=alt.X('sweetness:Q'),
    y = 'cuisine',
    color = color
    ).properties(width=300, height=300, title='Sweetness'
).transform_filter(
    selection
)

chart_spicy = alt.Chart(flavor_grouped_byCuisine).mark_bar(
    ).encode(
        x=alt.X('spiciness:Q'),
    y = 'cuisine',
    color = color
    ).properties(width=300, height=300, title='Spiciness'
).transform_filter(
    selection
)
    
chart_salty = alt.Chart(flavor_grouped_byCuisine).mark_bar(
    ).encode(
        x=alt.X('saltiness:Q'),
    y = 'cuisine',
    color = color
    ).properties(width=300, height=300, title='Saltiness'
).transform_filter(
    selection
)

chart_sour = alt.Chart(flavor_grouped_byCuisine).mark_bar(
    ).encode(
        x=alt.X('sourness:Q'),
    y = 'cuisine',
    color = color
    ).properties(width=300, height=300, title='Sourness'
).transform_filter(
    selection
)

chart_bitter = alt.Chart(flavor_grouped_byCuisine).mark_bar(
    ).encode(
        x=alt.X('bitterness:Q'),
    y = 'cuisine',
    color = color
    ).properties(width=300, height=300, title='Bitterness'
).transform_filter(
    selection
)

chart_savory = alt.Chart(flavor_grouped_byCuisine).mark_bar(
    ).encode(
        x=alt.X('savoriness:Q'),
    y = 'cuisine',
    color = color
    ).properties(width=300, height=300, title='Savoriness'
).transform_filter(
    selection
)

chart_fatty = alt.Chart(flavor_grouped_byCuisine).mark_bar(
    ).encode(
        x=alt.X('fattiness:Q'),
    y = 'cuisine',
    color = color
    ).properties(width=300, height=300, title='Fattiness'
).transform_filter(
    selection
)

print("Hold down SHIFT and click on the boxes next to two or more cuisines to see how their average flavor profiles compare:")
alt.vconcat(make_selector, chart_sweet | chart_spicy | chart_fatty, chart_salty | chart_sour | chart_bitter | chart_savory)

Hold down SHIFT and click on the boxes next to two or more cuisines to see how their average flavor profiles compare:


alt.VConcatChart(...)